In [1]:
import sys, time, wmi, psutil
SYSTEM_INFO = wmi.WMI().Win32_OperatingSystem()[0]
"system: {0}, {1}, {2}".format(SYSTEM_INFO.Caption, SYSTEM_INFO.BuildNumber, SYSTEM_INFO.OSArchitecture) 
"memory: {}G".format(round(psutil.virtual_memory().total / 1024**3, 2))
"cpu: {}".format(psutil.cpu_count())
"python: {}".format(sys.version)
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'system: Microsoft Windows 10 教育版, 18363, 64 位'

'memory: 15.86G'

'cpu: 4'

'python: 3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]'

'2020-10-04 20:26:22'

- **@author**: run_walker
- **@references**:
    1. [官方文档 > sorted函数](https://docs.python.org/zh-cn/3.7/library/functions.html?#sorted)
    2. [官方文档 > 排序指南](https://docs.python.org/zh-cn/3.7/howto/sorting.html#sortinghowto)
    3. [`object.__lt__(self, other)`](https://docs.python.org/zh-cn/3.7/reference/datamodel.html#object.__lt__)
    4. [Timsort排序算法](https://www.cnblogs.com/sunshuyi/p/12680918.html)

In [2]:
?sorted

Signature: sorted(iterable, /, *, key=None, reverse=False)
Docstring:
Return a new list containing all items from the iterable in ascending order.

A custom key function can be supplied to customize the sort order, and the
reverse flag can be set to request the result in descending order.
Type:      builtin_function_or_method


- `list`有一个内置的`list.sort()`方法可以直接修改列表。
- `sorted()`内置函数，可以从一个**可迭代对象**构建一个新的排序的`list`。

主要区别在于`sorted`不仅限于排序`list`，而且返回一个新的`list`，不是inplace操作的。

# 传入函数

## 忽略字符串大小

In [3]:
sorted(['This', 'is', 'a', 'test', 'string', 'from', 'Andrew'], key=str.lower)

['a', 'Andrew', 'from', 'is', 'string', 'test', 'This']

## 按dict的value

In [4]:
d = {'a': 10, 'b': 5, 'c': 9}
sorted(d, key=d.get)

['b', 'c', 'a']

In [5]:
sorted(d, key=d.__getitem__)

['b', 'c', 'a']

## 按可索引对象的某个位置

In [6]:
from operator import itemgetter

sorted([('a', 10), ('b', 5), ('c', 9)], key=itemgetter(1))

[('b', 5), ('c', 9), ('a', 10)]

允许多级排序

In [7]:
sorted([('john', 'A', 15), ('dave', 'B', 10), ('jane', 'B', 12)], key=itemgetter(1, 2))

[('john', 'A', 15), ('dave', 'B', 10), ('jane', 'B', 12)]

## 按类的某个属性

In [8]:
class Student:
    def __init__(self, name, grade, age):
        self.name = name
        self.grade = grade
        self.age = age
    def __repr__(self):
        return str((self.name, self.grade, self.age))
    
students = [
    Student('john', 'A', 15),
    Student('jane', 'B', 12),
    Student('dave', 'B', 10)
]

In [9]:
from operator import attrgetter

sorted(students, key=attrgetter('age'))

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

允许多级排序

In [10]:
sorted(students, key=attrgetter('grade', 'age'))

[('john', 'A', 15), ('dave', 'B', 10), ('jane', 'B', 12)]

# 重构类的比较运算符`__lt__`
`x < y`调用`x.__lt__(y)`。

## 例1

In [19]:
Student.__lt__ = lambda self, other: self.age < other.age

sorted(students)

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

## 重构字符串的比较运算符
- [LeetCode 179. 最大数](https://leetcode-cn.com/problems/largest-number/)：给定一组非负整数，重新排列它们的顺序使之组成一个最大的整数。
- [剑指 Offer 45. 把数组排成最小的数](https://leetcode-cn.com/problems/ba-shu-zu-pai-cheng-zui-xiao-de-shu-lcof/)：输入一个非负整数数组，把数组里所有数字拼接起来排成一个数，打印能拼接出的所有数字中最小的一个。

注意：为避免影响到`str`的正常比较，定义一个子类继承`str`，在其上重构`__lt__()`

In [34]:
class NumStr(str):
    def __lt__(x, y):
        return x + y > y + x

def largest_number(nums):
    res = ''.join(sorted(map(str, nums), key=NumStr))  # key传入重构了__lt__方法的类名
    return '0' if res[0] == '0' else res

In [35]:
largest_number([10, 2])

'210'

In [36]:
largest_number([3, 30, 34, 5, 9])

'9534330'

In [37]:
largest_number([352, 35234])

'35235234'

# 排序稳定性
`sorted`是稳定的。

如果一个排序确保不会改变比较结果相等的元素的相对顺序就称其为**稳定的** --- 这有利于进行多重排序（例如先按部门、再按薪级排序）。

In [14]:
from operator import itemgetter

sorted([('a', 2), ('b', 2), ('a', 1), ('b', 1)], key=itemgetter(0))

[('a', 2), ('a', 1), ('b', 2), ('b', 1)]

In [15]:
sorted([('a', 2), ('b', 2), ('a', 1), ('b', 1)])

[('a', 1), ('a', 2), ('b', 1), ('b', 2)]

例如，要按grade降序然后age升序对学生数据进行排序：

In [16]:
s1 = sorted(students, key=attrgetter('age'))     # sort on secondary key
s2 = sorted(s1, key=attrgetter('grade'), reverse=True)

In [18]:
students
s1
s2

[('john', 'A', 15), ('jane', 'B', 12), ('dave', 'B', 10)]

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

# 排序复杂度
python中默认使用的排序算法为**Timsort算法**。